In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="gFYtUrZhLA0MakrBM9AW")
project = rf.workspace("brad-dwyer").project("wildfire-smoke")
dataset = project.version(1).download("yolov7")

In [5]:
from google.colab.patches import cv2_imshow
from scipy.signal import correlate2d
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

## Database analysis

In [6]:
%cd /content/Wildfire-Smoke-1

/content/Wildfire-Smoke-1


In [ ]:
def show_dataset_shape_info(path):
  obj = {}
  for file in os.listdir(path):
    if '.jpg' not in file:
      continue
    image = cv2.imread(path + '/' + file)

    key = str(image.shape)

    if key not in obj:
      obj[key] = 1
    else:
      obj[key] += 1

  print(obj)

In [ ]:
show_dataset_shape_info('test/images')  # test
show_dataset_shape_info('train/images') # train
show_dataset_shape_info('valid/images') # valid

{'(480, 640, 3)': 74}
{'(480, 640, 3)': 516}
{'(480, 640, 3)': 147}


In [ ]:
def check_images_without_label(path):
  print('Total files: ', len(os.listdir(path)))
  for file in os.listdir(path):
    if '.txt' not in file:
      continue
    file_opened = open(path + '/' + file)
    file_content = file_opened.read()

    if (len(file_content.split(' ')) != 5): # verifica se possui a bouding box
      print(file)

In [ ]:
check_images_without_label('test/labels')
check_images_without_label('train/labels')
check_images_without_label('valid/labels')

Total files:  74
Total files:  516
Total files:  147


In [ ]:
def create_unique_database():
  %cd /content/Wildfire-Smoke-1
  %mkdir database
  %mv /content/Wildfire-Smoke-1/test/images /content/Wildfire-Smoke-1/database/images
  %mv /content/Wildfire-Smoke-1/test/labels /content/Wildfire-Smoke-1/database/labels

  %mv /content/Wildfire-Smoke-1/train/images/* /content/Wildfire-Smoke-1/database/images
  %mv /content/Wildfire-Smoke-1/train/labels/* /content/Wildfire-Smoke-1/database/labels

  %mv /content/Wildfire-Smoke-1/valid/images/* /content/Wildfire-Smoke-1/database/images
  %mv /content/Wildfire-Smoke-1/valid/labels/* /content/Wildfire-Smoke-1/database/labels

create_unique_database()

/content/Wildfire-Smoke-1


In [ ]:
show_dataset_shape_info('database/images')  # database
check_images_without_label('database/labels')

{'(480, 640, 3)': 737}
Total files:  737


## Pre-processing

In [ ]:
def flip_horizontal(image, label):  
  image_flip = cv2.flip(image, 1)

  label_flip = label.copy()
  label_flip[:, 1] = (1 - label[:, 1])

  return image_flip, label_flip

In [ ]:
def flip_vertical(image, label):  
  image_flip = cv2.flip(image, 0)

  label_flip = label.copy()
  label_flip[:, 2] = (1 - label[:, 2])

  return image_flip, label_flip

In [ ]:
def brightness(image, label, alpha=1.0, beta=60.0):  
  image_brightness = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

  return image_brightness, label.copy()

In [ ]:
def saturation(image, label, saturation_factor=50):
  hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  hsv_img[:,:,1] = np.clip(hsv_img[:,:,1] + saturation_factor, 0, 255)
  saturate_image = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)

  return saturate_image, label.copy()

In [ ]:
def smoothing(image, label, kernel_size = (5, 5)):   
  blurred_img = cv2.GaussianBlur(image, kernel_size, 0)

  return blurred_img, label.copy()

In [ ]:
def save_image_and_label(image, image_full_path, label, label_full_path):
  cv2.imwrite(image_full_path + '.jpg', image)
  np.savetxt(label_full_path + '.txt', label, fmt=["%d", "%.6f", "%.6f", "%.6f", "%.6f"])

In [ ]:
def plot_images(images, titles):
  fig = plt.figure(figsize=(10, 8))
  axes = fig.subplots(nrows=2, ncols=3)

  for i, ax in enumerate(axes.flat):
      if i < len(images):
          rgb_image = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
          ax.imshow(rgb_image)
          ax.set_title(titles[i])

      ax.set_axis_off()
  
  plt.subplots_adjust(hspace=0.001, wspace=0.1)
  plt.show()

In [ ]:
def data_augmentation(image_path, label_path, processing = []):
  original_images_list = os.listdir(image_path)
  for file in original_images_list:
    if '.jpg' not in file:
      continue
    
    image = cv2.imread(image_path + '/' + file)
    label = np.loadtxt(label_path + '/' + file.split('.jpg')[0] + '.txt').reshape(-1, 5)
    
    base_img_path = image_path + '/' + file.split('.jpg')[0]
    base_label_path = label_path + '/' + file.split('.jpg')[0]

    img = [image]
    titles = ['original']
    for processing_technique in processing:      
      if processing_technique == 'flip_horizontal':
        new_image, new_label = flip_horizontal(image, label)
      elif processing_technique == 'flip_vertical':
        new_image, new_label = flip_vertical(image, label)
      elif processing_technique == 'brightness':
        new_image, new_label = brightness(image, label)
      elif processing_technique == 'saturation':
        new_image, new_label = saturation(image, label)
      elif processing_technique == 'smoothing':
        new_image, new_label = smoothing(image, label)
      else:
        continue
      
      img.append(new_image)
      titles.append(processing_technique)
      save_image_and_label(new_image, base_img_path + '.' + processing_technique, new_label, base_label_path + '.' + processing_technique)
      
    #plot_images(img, titles)
    #break

In [ ]:
data_augmentation('database/images', 'database/labels', ['flip_horizontal', 'flip_vertical', 'brightness', 'saturation', 'smoothing'])

In [ ]:
show_dataset_shape_info('database/images') # conferir a nova quantidade de imagem e os formatos

{'(480, 640, 3)': 4422}


In [ ]:
check_images_without_label('database/labels') # conferir a nova quantidade de labels

Total files:  4422


## Exploration

In [8]:
def view_image(path):
  for file in os.listdir(path):
    image = cv2.imread(path + '/' + file)

    cv2_imshow(image)
    cv2.waitKey(0)

In [10]:
view_image('test/images')

FileNotFoundError: ignored

In [ ]:
# unused - wrong implementation cause Pearson correlation it is a better choice for categorical 
def coorelation_pearson(path):
  min_corr = {
      'image1': '',
      'image2': '',
      'cor': 1,
  }

  files_checked = []
  for file1 in os.listdir(path):
    if ('.jpg' not in file1):
      continue
    image1 = cv2.imread(path + '/' + file1)
    files_checked.append(file1)

    for file2 in os.listdir(path):
      if '.jpg' not in file2:
        continue

      if file2 in files_checked:
        continue

      image2 = cv2.imread(path + '/' + file2)

      image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
      image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

      correlacao = np.corrcoef(image1_gray.flatten(), image2_gray.flatten())[0, 1]

      if correlacao < min_corr['cor']:
        print(correlacao)
        min_corr['cor'] = correlacao
        min_corr['image1'] = file1
        min_corr['image2'] = file2

      #if correlacao < 0:
      #  print(file1, '->' , file2)

  print(min_corr)

coorelation_pearson('database/images')

In [ ]:
def cross_correlation(path):
  image_base = cv2.imread('/content/Wildfire-Smoke-1/database/images/ck0k99e6p79go0944lmxivkmv_jpeg.rf.035851b1218826a8d8e1f6cff055da7a.flip_horizontal.jpg', 0)

  arr = []
  mean_arr = []
  for i, file in enumerate(os.listdir(path)):
      image = cv2.imread(path + '/' + file, 0)

      fft_imagem1 = np.fft.fft2(image_base)
      fft_imagem2 = np.fft.fft2(image)

      correlacao = np.fft.ifft2(fft_imagem1 * np.conj(fft_imagem2))
      correlacao = abs( 255 - ((np.abs(correlacao) / np.max(correlacao)) * 255))

      arr.append(correlacao)
      
      if i % 500 == 0:
        mean_arr.append(list(np.mean(np.array(arr), axis=0)))
        arr = []

  cv2_imshow(np.mean(np.array(mean_arr), axis=0))

cross_correlation('database/images')

In [ ]:
def calculate_hist_mean(path):
  num_bins = 256
  hist_sum = np.zeros(num_bins)
  image_path = os.listdir(path)

  for file in image_path:
      image = cv2.imread(path + '/' + file)
      
      image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      hist = cv2.calcHist([image_gray], [0], None, [num_bins], [0, num_bins])
      
      hist_sum += hist.flatten()

  total_images = len(image_path)
  hist_mean = hist_sum / total_images
  plt.plot(hist_mean)

  plt.title('Histograma Médio')
  plt.xlabel('Intensidade de Pixel')
  plt.ylabel('Frequência')
  plt.show()

In [ ]:
calculate_hist_mean('database/images')

In [ ]:
def get_labels_in_list(path, image_width, image_height):
  bouding_boxes = []
  for file in os.listdir(path):
    file_opened = open(path + '/' + file)
    file_content = file_opened.read()
    
    x, y, w, h = list(map(float, file_content.split(' ')[1:]))
    x = x * image_width
    y = y * image_height
    w = w * image_width
    h = h * image_height

    left = int(x - w / 2)
    top = int(y - h / 2)
    right = int(x + w / 2)
    bottom = int(y + h / 2)
    
    bouding_boxes.append([left, top, right, bottom])

  return bouding_boxes

rectangles = np.zeros_like(np.empty((480, 640, 3)))[:, :, 0].astype('float32')
for row in get_labels_in_list('database/labels', 640, 480):
    l, t, r, b = row
    rectangles[int(t):int(b), int(l):int(r)] += 1
    
rectangles[rectangles == 0] = np.nan

plt.imshow(rectangles, alpha=0.5)

## Configuração da YoloV7

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7
%cd /content/yolov7
!pip install -r requirements.txt

## Fine-tuning based in Coco training

In [ ]:
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

Treinamento

In [ ]:
%cd /content/yolov7
!python train.py --batch 16 --epochs 20 --data '/content/smoke-detection-2/data.yaml' --weights 'yolov7_training.pt' --device 0 